#Advanced Minimax Implementation with Alpha-Beta Pruning: By Fransi Ayele

In [ ]:
import math
import random
import time

# --- CONSTANTS ---
X_PLAYER = "X"  # The Maximizing Player (AI)
O_PLAYER = "O"  # The Minimizing Player (Human)
EMPTY = " "

class TicTacToeGame:
    """
    Handles the Game State, rules, and board manipulation.
    This separates the 'Rules' from the 'Intelligence'.
    """
    def __init__(self):
        self.board = [EMPTY] * 9
        self.winner = None

    def get_valid_moves(self):
        """Returns a list of indices (0-8) that are empty."""
        return [i for i, x in enumerate(self.board) if x == EMPTY]

    def make_move(self, index, player):
        """Places a marker on the board if the move is valid."""
        if self.board[index] == EMPTY:
            self.board[index] = player
            return True
        return False

    def undo_move(self, index):
        """Reverts a move (crucial for the backtracking step in Minimax)."""
        self.board[index] = EMPTY
        self.winner = None # Reset winner status when backtracking

    def check_winner(self, player):
        """Checks if the specific player has won."""
        # Pre-defined winning indices for efficiency
        win_conditions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8), # Rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8), # Cols
            (0, 4, 8), (2, 4, 6)             # Diagonals
        ]
        for a, b, c in win_conditions:
            if self.board[a] == self.board[b] == self.board[c] == player:
                self.winner = player
                return True
        return False

    def is_draw(self):
        """Checks if the game is a draw (full board, no winner)."""
        return EMPTY not in self.board and not self.winner

    def is_game_over(self):
        """Returns True if X wins, O wins, or Draw."""
        return self.check_winner(X_PLAYER) or self.check_winner(O_PLAYER) or self.is_draw()

    def print_board(self):
        """Renders the board with a coordinate guide."""
        b = self.board
        print("\n  Current Board          Reference Map")
        print(f"  {b[0]} | {b[1]} | {b[2]}            0 | 1 | 2 ")
        print(" ---+---+---          ---+---+---")
        print(f"  {b[3]} | {b[4]} | {b[5]}            3 | 4 | 5 ")
        print(" ---+---+---          ---+---+---")
        print(f"  {b[6]} | {b[7]} | {b[8]}            6 | 7 | 8 \n")


class MinimaxAgent:
    """
    The Intelligence unit.
    Uses Minimax with Alpha-Beta Pruning and Depth-Scaled scoring.
    """
    def __init__(self, player_symbol):
        self.player = player_symbol

    def evaluate(self, game, depth):
        """
        The Heuristic Utility Function.
        Returns +10 for AI win, -10 for Human win, 0 for Draw.
        Subtracts 'depth' to prefer winning sooner.
        """
        if game.check_winner(X_PLAYER):
            return 10 - depth  # Win sooner = Higher Score
        elif game.check_winner(O_PLAYER):
            return depth - 10  # Lose later = Higher Score (less negative)
        return 0

    def minimax(self, game, depth, is_maximizing, alpha, beta):
        # 1. Base Case: Check if game is over
        if game.is_game_over():
            return self.evaluate(game, depth)

        # 2. Recursive Step
        if is_maximizing:
            max_eval = -math.inf
            for move in game.get_valid_moves():
                game.make_move(move, X_PLAYER)
                # Recursively call minimax for the opponent
                eval_score = self.minimax(game, depth + 1, False, alpha, beta)
                game.undo_move(move) # Backtrack!

                max_eval = max(max_eval, eval_score)
                alpha = max(alpha, eval_score) # Update Alpha

                # Alpha-Beta Pruning
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = math.inf
            for move in game.get_valid_moves():
                game.make_move(move, O_PLAYER)
                # Recursively call minimax for the AI
                eval_score = self.minimax(game, depth + 1, True, alpha, beta)
                game.undo_move(move) # Backtrack!

                min_eval = min(min_eval, eval_score)
                beta = min(beta, eval_score) # Update Beta

                # Alpha-Beta Pruning
                if beta <= alpha:
                    break
            return min_eval

    def get_best_move(self, game):
        """
        Root of the Minimax search.
        Iterates through all possible first moves to find the optimal path.
        """
        best_score = -math.inf
        best_moves = [] # List to store multiple equally good moves

        # Optimization: If it's the very first move, take center (4) or a corner.
        # This saves computing the entire tree for an empty board.
        if len(game.get_valid_moves()) == 9:
            return 4

        print(f"AI ({self.player}) is calculating optimal move...")
        start_time = time.time()

        for move in game.get_valid_moves():
            game.make_move(move, self.player)
            # Start Minimax (next turn is minimizer)
            score = self.minimax(game, 0, False, -math.inf, math.inf)
            game.undo_move(move)

            if score > best_score:
                best_score = score
                best_moves = [move] # Found a new best score, reset list
            elif score == best_score:
                best_moves.append(move) # Found another move just as good

        end_time = time.time()
        print(f"Calculation time: {round(end_time - start_time, 4)}s | Nodes Evaluated: Full Tree Pruned")

        # Randomly select among the best moves to make the AI feel less robotic
        return random.choice(best_moves) if best_moves else -1

# --- MAIN EXECUTION BLOCK ---
def play_game():
    game = TicTacToeGame()
    ai = MinimaxAgent(X_PLAYER)

    print("-------------------------------------------------")
    print("  Minimax Agent: Tic-Tac-Toe (Alpha-Beta Pruned) ")
    print("-------------------------------------------------")

    # Randomly decide who starts to fulfill 'Optional Enhancements'
    current_turn = X_PLAYER if random.choice([True, False]) else O_PLAYER
    print(f"Coin toss result: {current_turn} goes first!")

    while not game.is_game_over():
        game.print_board()

        if current_turn == O_PLAYER:
            # Human Turn
            try:
                move = int(input(f"Your Turn ({O_PLAYER}). Choose 0-8: "))
                if move not in game.get_valid_moves():
                    print(">> Invalid move! Cell occupied or out of range.")
                    continue
                game.make_move(move, O_PLAYER)
                current_turn = X_PLAYER
            except ValueError:
                print(">> Please enter a valid integer.")
        else:
            # AI Turn
            move = ai.get_best_move(game)
            game.make_move(move, X_PLAYER)
            print(f"AI chooses position: {move}")
            current_turn = O_PLAYER

    # Final State
    game.print_board()
    if game.check_winner(X_PLAYER):
        print("Result: AI Wins! Better luck next time.")
    elif game.check_winner(O_PLAYER):
        print("Result: You Win! (This is theoretically impossible).")
    else:
        print("Result: It's a Draw! (Perfect Play)")

if __name__ == "__main__":
    play_game()

-------------------------------------------------
  Minimax Agent: Tic-Tac-Toe (Alpha-Beta Pruned) 
-------------------------------------------------
Coin toss result: X goes first!

  Current Board          Reference Map
    |   |              0 | 1 | 2 
 ---+---+---          ---+---+---
    |   |              3 | 4 | 5 
 ---+---+---          ---+---+---
    |   |              6 | 7 | 8 

AI chooses position: 4

  Current Board          Reference Map
    |   |              0 | 1 | 2 
 ---+---+---          ---+---+---
    | X |              3 | 4 | 5 
 ---+---+---          ---+---+---
    |   |              6 | 7 | 8 

Your Turn (O). Choose 0-8: 0

  Current Board          Reference Map
  O |   |              0 | 1 | 2 
 ---+---+---          ---+---+---
    | X |              3 | 4 | 5 
 ---+---+---          ---+---+---
    |   |              6 | 7 | 8 

AI (X) is calculating optimal move...
Calculation time: 0.0042s | Nodes Evaluated: Full Tree Pruned
AI chooses position: 7

  Current